<a href="https://colab.research.google.com/github/vanderbilt-data-science/AIDA/blob/13-create-beta-version-front-end-gradio/50_frontend_testing_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 50-frontend-testing-gradio

In this notebook, we will delve into the implementation of Gradio, covering both its general usage and more advanced features. We will explore how to incorporate features like file upload, interaction with generative AI, and configuring parameters that are essential for our AIDA project.

## Install & load packages

In [1]:
%%capture
!pip install -q gradio
!pip install -q transformers
!pip install -q langchain
!pip install -q openai


In [2]:
import gradio as gr

## Test gradio

This section introduces the basic Gradio function as a starting point before we proceed to explore more advanced features.

In [3]:
def greet(name, isMorning, temperature):
    salutation = "Good morning" if isMorning else "Good evening"
    greeting = "%s %s. It is %s degrees today" % (salutation, name, temperature)
    celsius = (temperature - 32) * 5 / 9
    return greeting, round(celsius, 2)

demo = gr.Interface(
    fn=greet,
    inputs=[gr.Textbox(lines=2, placeholder="Your Name Here..."), gr.Checkbox(label="Is it Morning?"), gr.Slider(0, 100)],
    outputs=[gr.Textbox(label = "Greeting"), gr.Number(label = "Temperature in Celsius")],
    examples=[
        ["Umang", True, 65],
        ["Jesse", False, 95]]
)
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## First version -- file upload


This code snippet requires the user to upload a file, reads the contents of the file, and generates an output based on the incorporated parameters. (Note that the generated output is intended for display purposes and may not be accurate since accurate prompts haven't been provided.)

In [4]:
def read_story(story_file):
    with open(story_file.name, "r") as f:
        story = f.read()
    return story

def ask_questions(story, prompt, count, age):
    print("Please provide the story:")
    print(story)
    print("Paste user guide prompt:")
    print(prompt)
    print("How many times have you read the story?")
    print(count)
    print("What's the reader's age?")
    print(age)

story_input = gr.inputs.File(label="Upload Story")
prompt_input = gr.inputs.Textbox(label="User Guide Prompt")
count_input = gr.inputs.Number(label="The number of times users read the story")
age_input = gr.inputs.Number(label="Reader's Age")
interface = gr.Interface(fn=ask_questions,
                         inputs=[story_input, prompt_input, count_input, age_input],
                         outputs='text')
interface.launch()

<ipython-input-4-84c952356c51>:16: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  story_input = gr.inputs.File(label="Upload Story")
<ipython-input-4-84c952356c51>:16: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  story_input = gr.inputs.File(label="Upload Story")
<ipython-input-4-84c952356c51>:16: GradioDeprecationWarning: `keep_filename` parameter is deprecated, and it has no effect
  story_input = gr.inputs.File(label="Upload Story")
<ipython-input-4-84c952356c51>:17: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  prompt_input = gr.inputs.Textbox(label="User Guide Prompt")
<ipython-input-4-84c952356c51>:17: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  prompt_input = gr.inputs.Textbox(l

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## Second version -- text with transformers


In this section, we will leverage a generative AI model, specifically the Transformers, to generate the output that can be integrated with our prompts.

In [8]:
# Set OpenAI API Key
# Run this code and paste the API key and press 'enter'

import os
import openai
from getpass import getpass

openai_api_key = getpass()
os.environ["OPENAI_API_KEY"] = openai_api_key

··········


In [10]:
# In some cases, the above code may not work. . In such instances, you have the option to directly input your API key by replacing 'sk-...' with your own API key.
# openai.api_key = 'sk-..'


In [11]:
# test the transformer model
completion = openai.ChatCompletion.create(
  model = "gpt-3.5-turbo",
  temperature = 0.8,
  max_tokens = 2000,
  messages = [
    {"role": "system", "content": "You are a poet who creates poems that evoke emotions."},
    {"role": "user", "content": "Write a short poem for programmers."}
  ]
)

print(completion.choices[0].message.content)

In lines of code, a world unfolds,
Where logic reigns, and dreams take hold.
With keystrokes swift, and minds ablaze,
Programmers craft, a digital maze.

Through algorithms, paths are sought,
Seeking solutions, with every thought.
From bugs to fix, to functions refined,
They weave a tapestry, of code entwined.

Late nights spent, in code's embrace,
Creating wonders, in virtual space.
With syntax sharp, and bugs laid bare,
They strive for perfection, beyond compare.

For in this realm, where lines connect,
Programmers build, what others neglect.
Their fingertips dance, with grace profound,
Creating magic, without a sound.

So let us honor, these masters of code,
Whose creations shape, the future's ode.
In their hands, technology thrives,
With logic, passion, and infinite drives.


In [12]:
# The predict function utilizes the inputs from the Gradio interface,
# along with the prompts provided, to generate an output.

def predict(story, prompt, count, age, level, language):
    # Using the OpenAI API, we create a completion
    # by sending a request with the specified parameters.
    response = openai.Completion.create(
        engine="text-davinci-002",  # GPT-3.5 engine
        prompt= prompt,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.7,
    )
    completion = response.choices[0].text
    return completion

## Last version -- setting up the Gradio interface for the AIDA application


The following code accepts inputs such as a story, prompts, the number of times the story has been read, the child's age, the child's reading level, and the language. It generates an output that is relevant to the specified parameters. Please note that specific prompts have not been predetermined in this stage. For now, users can simply input prompts like 'generate 3 questions related to the story'.

In [13]:
title = "AIDA(Artificially Intelligent Dialogic Reading Aid)"

description = """
A simple application that takes in inputs from the book and returns a questions
for children to check their understanding of the book.
"""

#article = "Check out more documentation at https://www.gradio.app/getting_started/#getting_started"


story_input = gr.inputs.Textbox(lines = 10, label="Upload a Story")
prompt_input = gr.inputs.Textbox(label="User Guide Prompt")
count_input = gr.inputs.Number(label="The number of time children read the story")
age_input = gr.inputs.Number(label="Reader's Age")
level_input = gr.inputs.Number(label="Reader's Reading Level (by age)")
language_input = gr.Dropdown(label = "Language", choices = ["English", "Spanish"])

question_output = gr.Textbox(label = "Generated Questions")
demo = gr.Interface(
    fn=predict,
    inputs=[story_input, prompt_input, count_input,
            age_input,level_input, language_input],
    outputs= question_output,
    title=title,
    description=description,
    theme="compact",
    layout="vertical"
)

demo.launch()

<ipython-input-13-e0f1d438714a>:11: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  story_input = gr.inputs.Textbox(lines = 10, label="Upload a Story")
<ipython-input-13-e0f1d438714a>:11: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  story_input = gr.inputs.Textbox(lines = 10, label="Upload a Story")
<ipython-input-13-e0f1d438714a>:11: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  story_input = gr.inputs.Textbox(lines = 10, label="Upload a Story")
<ipython-input-13-e0f1d438714a>:12: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  prompt_input = gr.inputs.Textbox(label="User Guide Prompt")
<ipython-input-13-e0f1d438714a>:12: GradioDeprecationWarning: `optional` parameter is deprecated, and it 

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>